In [52]:
import json
import numpy as np
import pandas as pd
from nltk.stem.snowball import FrenchStemmer
import unicodedata
import re
import math

In [53]:
df=pd.read_csv("Data/commune2021.csv")
communes_ = np.array(df["LIBELLE"])
_insee_communes, counts = np.unique(communes_, return_counts=True)
insee_communes = np.unique([x.lower() for x in _insee_communes])

In [54]:
_dbPedia = json.load(open("Data/dbPedia_extracted_demonyms3.json"))
dbPedia = { x.lower():v for x,v in _dbPedia.items()}
_infobox = json.load(open("Data/infobox_extracted_demonyms3.json"))
infobox = { x.lower():v for x,v in _infobox.items()}
habitants_com =  json.load(open("Data/habitants_com_extracted_demonyms4.json"))

In [55]:
merged = {}
for c in insee_communes:
    s1 = set([ x.lower() for x in dbPedia.get(c, [])])
    s2 = set([x.lower() for x in infobox.get(c, [])])
    s3 = set([x.lower() for x in habitants_com.get(c, [])])
    s = s1.union(s2).union(s3)
    if len(s)>0:
        merged[c] = list(s)
    

In [56]:
len(merged)

30156

In [57]:
# with open("Data/merged.json", "w") as fp:
#         json.dump(merged,fp)

In [58]:
def stem_nltk(word):
    stemmer = FrenchStemmer()
    return stemmer.stem(word)

In [59]:
all = json.load(open("Data/merged.json"))
reversed , reversed_stemmed = {}, {}

for commune, demonyms in all.items():
    for demonym in demonyms:
        if demonym in reversed:
            reversed[demonym].add(commune)
        else:
            reversed[demonym] = set([commune])

        stemmed = stem_nltk(demonym)
        if stemmed in reversed_stemmed:
            reversed_stemmed[stemmed].add(commune)
        else:
            reversed_stemmed[stemmed] = set([commune])
for k,v in reversed.items():
    reversed[k] = list(v)
for k,v in reversed_stemmed.items():
    reversed_stemmed[k] = list(v)

with open("Data/merged_reversed.json", "w") as fp:
         json.dump(reversed,fp)
with open("Data/merged_reversed_stemmed.json", "w") as fp:
         json.dump(reversed_stemmed,fp)

In [60]:
def get_location_demonym_dict(demonym, stemmed_dict):
    stemmed = stem_nltk(demonym)
    if stemmed in stemmed_dict:
        return stemmed_dict[stemmed]
    return -1

In [61]:
get_location_demonym_dict("bellifontaines", reversed_stemmed)

['beaulieu-les-fontaines', 'belfonds', 'bellefontaine', 'fontainebleau']

In [62]:
print(len(reversed), len(reversed_stemmed))

61689 39568
